# squat deadlift regression
The below notebook will show the relationship between back squat and deadlift for CrossFit athletes based on the information provided by their athlete profiles.

## imports

In [7]:
import pandas as pd
import pymysql as pms

# use for implementing custom modules
import sys
sys.path.insert(0, "../custom_modules/")

#custom modules
from dataframer import grab_data